In [1]:
import ee
import datetime
import folium
import os
import itertools
import sys

from pprint import pprint
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

In [3]:
# append the parent path into sys-path so wen can import necessary modules
sys.path.append('../')

In [4]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [5]:
from BackGround_modules.Class_1_Make_fourier_imgs import Make_Fourier
from BackGround_modules.Class_2_Classify_Fourier_Img import Classification
from BackGround_modules.Class_3_Calculate_the_accuracy import Accuracy_assesment

The default supplementary data a Zero img, change accordingly when classify other years imgs



### Step_0_Define_Basic_Parameters

In [6]:
# Define the time-range
start_time = '2017-01-01'
end_time   = '2019-12-31'

# Create the year_name for naming variables
year_name = f'{start_time[:4]}_{end_time[:4]}'

# Define the interval for random classifier tree_num test
Interval_index = [1] + list(range(0,121,10))[1:]

In [7]:
# import some spatial constrains
North_china_plain_boundary = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Boundary_shp/North_China_Plain_Boundary")

# import verified points
Verified_Point   = ee.FeatureCollection(f"users/wangjinzhulala/North_China_Plain_Python/Sample_Points/Verified_pt_{year_name}")
Verified_Point_with_city_rural = ee.FeatureCollection(f"users/wangjinzhulala/North_China_Plain_Python/Sample_Points/Verified_pt_{year_name}_with_20K_city_rural_pts");

### Step_1_Determine the best Harmonic number

In [9]:
# use if condition to find out the right Landsat img formulation
# here use Landsat Raw data because the simplecloudComposite 
# algorithm can be easily applied to it.

# use the end_year as threshold
end_year = int(year_name.split('_')[1])

if  end_year  <= 2010:
    Landsat_img = ee.ImageCollection("LANDSAT/LT05/C01/T1")

elif end_year <= 2013:
    Landsat_img = ee.ImageCollection("LANDSAT/LE07/C01/T1")

else:
    Landsat_img = ee.ImageCollection("LANDSAT/LC08/C01/T1")

In [11]:
# Extract the Fourier_pixel_value to sample points
# 1) using year_range to make Fourier_img
# 2) for each Forier_img, try harmonic number of [1-5]


    
for harmonic in range(1,6):

    # define the asset path for exporting
    asset_bucket = 'gs://north_china_plain/Process_2_GEE_Python_Classification/Sub_Process_1_Determine_best_(Year_range,Fourier_num,Tree_num)'
    # define the name for exporting
    extract_name = f'Fourier_img_{year_name}_harmonic_{harmonic}'


    # make the Fourier img, the default is to make time series
    # of ['NDVI', 'NDBI', 'EVI']
    Fourier_making = Make_Fourier( start_date = start_time,
                                   end_date   = end_time,
                                   harmonics  = harmonic )

    # Go through the Fourier img making process
    Fourier_making.Stp_1_Create_hamonic_names()
    Fourier_making.Stp_2_Add_harmonics()
    Fourier_making.Stp_3_Harmonic_fit()

    # Get the Fourier img
    Fourier_img = Fourier_making.harmonicTrendCoefficients

    # export the Fourier img
#     task = ee.batch.Export.image.toCloudStorage( 
#                                                 image          = Fourier_img,
#                                                 description    = f'Exporting {extract_name}',
#                                                 bucket         = asset_bucket,
#                                                 fileNamePrefix = extract_name,
#                                                 region         = North_china_plain_boundary.geometry().bounds().getInfo()['coordinates'],
#                                                 scale          = 30,
#                                                 maxPixels      = int(1e13),
#                                                 skipEmptyTiles = True)
#     task.start()

    # print out the process
    print(f'Exporting {extract_name}')

Analyzing the images of 2017_2019
Exporting Fourier_img_2017_2019_harmonic_1
Analyzing the images of 2017_2019
Exporting Fourier_img_2017_2019_harmonic_2
Analyzing the images of 2017_2019
Exporting Fourier_img_2017_2019_harmonic_3
Analyzing the images of 2017_2019
Exporting Fourier_img_2017_2019_harmonic_4
Analyzing the images of 2017_2019
Exporting Fourier_img_2017_2019_harmonic_5
